In [9]:
import json
import os
import pandas as pd

In [3]:
contents = os.listdir("data")

In [5]:
all_data = []
for x in contents:
    if ".json" not in x:
        continue
    with open(f"data/{x}", "r") as f:
        part_data = json.load(f)
        all_data += part_data

In [7]:
len(all_data)

1252

In [8]:
with open("all_data.json", "w") as f:
    json.dump(all_data, f)

In [11]:
df = pd.DataFrame.from_dict(all_data)

In [12]:
df.head()

,meeting_type,location,datetime_iso,links
0,Peterborough Environmental Advisory Committee,Electronic Meeting,2021-11-17T17:30:00,[{'aria_label': 'Peterborough Environmental Ad...
1,Peterborough Environmental Advisory Committee,Electronic Meeting,2021-10-20T17:30:00,[{'aria_label': 'Peterborough Environmental Ad...
2,Peterborough Environmental Advisory Committee,Electronic Meeting,2021-09-15T17:30:00,[{'aria_label': 'Peterborough Environmental Ad...
3,Peterborough Environmental Advisory Committee,Electronic Meeting,2021-06-16T17:30:00,[{'aria_label': 'Peterborough Environmental Ad...
4,Peterborough Environmental Advisory Committee,Electronic Meeting,2021-05-19T17:30:00,[{'aria_label': 'Peterborough Environmental Ad...


In [14]:
df["links"][0]

[{'aria_label': 'Peterborough Environmental Advisory Committee November 17, 2021',
  'href': 'Meeting.aspx?Id=0ef82589-a943-4de4-adf7-4c69b5a685a7&lang=English',
  'link_text': 'Peterborough Environmental Advisory Committee',
  'url': 'https://pub-peterborough.escribemeetings.com/Meeting.aspx?Id=0ef82589-a943-4de4-adf7-4c69b5a685a7&lang=English'},
 {'aria_label': 'Agenda (HTML) for Peterborough Environmental Advisory Committee 17 November 2021',
  'href': 'Meeting.aspx?Id=0ef82589-a943-4de4-adf7-4c69b5a685a7&Agenda=Agenda&lang=English',
  'link_text': 'HTML',
  'url': 'https://pub-peterborough.escribemeetings.com/Meeting.aspx?Id=0ef82589-a943-4de4-adf7-4c69b5a685a7&Agenda=Agenda&lang=English'}]

In [20]:
# the first link in the list is always the agenda
df["agenda_url"] = df["links"].map(lambda x: x[0]["url"])

In [21]:
df["agenda_url"][100]

'https://pub-peterborough.escribemeetings.com/Meeting.aspx?Id=9c4fcc83-a9da-40c2-9ac0-91bf27402855&lang=English'

In [27]:
links = df["links"][10]

In [70]:
# if minutes is in aria_label.lower() then it is the minutes
# link text gives file type
def get_minues_url(links, file_type="html"):
    "file_type in ['html', 'pdf']"
    for x in links:
        if "minutes" not in x["aria_label"].lower():
            continue
        if file_type == x["link_text"].lower():
            return x["url"]
        return ""
    return ""

def is_cancelled(links):
    for x in links:
        if "cancellation" in x["link_text"].lower():
            return True
    return False

def is_cancelled(links):
    for x in links:
        if "cancellation" in x["link_text"].lower():
            return True
    return False

def get_video_url(links):
    for x in links:
        if x["link_text"].lower() == "video":
            return x["url"]
    return ""

In [71]:
df["minutes_html_url"] = df["links"].map(lambda x: get_minues_url(x, "html"))
df["minutes_pdf_url"] = df["links"].map(lambda x: get_minues_url(x, "pdf"))
df["cancelled"] = df["links"].map(is_cancelled)
df["video_url"] = df["links"].map(get_video_url)

In [74]:
df.columns

Index(['meeting_type', 'location', 'datetime_iso', 'links', 'agenda_url',
       'minutes_html_url', 'minutes_pdf_url', 'cancelled', 'video_url'],
      dtype='object')

In [75]:
cols_of_interest = ['meeting_type', 'location', 'datetime_iso', 'agenda_url', 'minutes_html_url', 'minutes_pdf_url', 'cancelled', 'video_url']

In [83]:
df[cols_of_interest].to_json("all_data_flat.json", orient="records")

In [89]:
all_meeting_types = list(set(df["meeting_type"]))

In [90]:
with open("data/all_meeting_types.json", "w") as f:
    json.dump(all_meeting_types, f)